<a href="https://colab.research.google.com/github/SydanJainen/4EU-DeepLearningCourse/blob/main/Assignments/AssignementCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignments

### We are given a dataset containing 510 images which are categorized according two main characteristics/properties: “C1” and “C2”. The following inputs are available:
* #### `input image', a 110 × 110 × 3 real-valued tensor. The last dimension denotes the number of input channels; the images belong to different combinations of the two properties C1 and C2. For C1 and C2 we have 5 and 6 possible values, respectively;  among the 30 possible couples (C1,C2), we have images for only 24 of them. We have around 20 samples (little more or less) for each of the available couples;  values in each tensor entry are integers in [0, 255];
* #### C1, string, the value for the C1 property. 5 possible values;
* #### C2, string, the value for the C2 property. 6 possible values.

### Design a deep neural network model to predict the class of an image, given by the couple (C1,C2).

### Provide a sketch of each of the following points, then implement your solution.
1. MODEL: Which architecture do you consider the most appropriate for this task, and why;
2. INPUT:
* After a potential preprocessing step, which is the input of the model, and
how is it represented;
3. OUTPUT: How would you design the output layer and why;
4. LOSS: Which loss function would you use to train your model and why;
5. MODEL CONFIGURATION:
 Model composition (composition of layers, regardless their number,
or their dimension, which can be object of tuning)
6. MODEL EVALUATION: How would you assess (in which setting) the
generalization capabilities of the model on unseen data?


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## go to your datapath
%cd /content/drive/MyDrive/INSEGNAMENTI/4EU+/

# Here you should see the desired files
%ls

ModuleNotFoundError: No module named 'google.colab'

In [13]:
import pickle as pk
# Open the pickle data
with open('input.pkl', 'rb') as f:
  data = pk.load(f)

images = data['imgs']
C1 = data['C1']
C2 = data['C2']
print(f"length image:{len(images)}")
print(f"length C1:{len(C1)}")
print(f"length C2:{len(C2)}")

length image:510
length C1:510
length C2:510


In [14]:
#import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import transforms


import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import random_split

In [15]:
transofm = Compose([ToTensor(),
                        Normalize((0,), (1,))])
tensor_images = []
for i in images :
  tensor_images.append(transofm(i))

In [16]:
for a in np.unique(C1):
  print(a)

dress
pants
shirt
shoes
shorts


In [17]:
for a in np.unique(C2):
  print(a)

black
blue
brown
green
red
white


In [18]:
main_dataset = zip(tensor_images, C1, C2)

label_combinations = {}
for i, (c1, c2) in enumerate(zip(C1, C2)):
    label_combinations[i] = f"{c1} {c2}"

unique_labels = set(label_combinations.values())
label_to_index = {label: i for i, label in enumerate(unique_labels)}
print(label_to_index)

enumerated_dataset = []
for i, (tensor_images, c1, c2) in enumerate(zip(tensor_images, C1, C2)):
  label_combination = f"{c1} {c2}"
  label_index = label_to_index[label_combination]
  enumerated_dataset.append((tensor_images, label_index))

print(len(enumerated_dataset))
#print(len(main_dataset))

#one_hot = torch.nn.functional.one_hot(enumerated_dataset)

{'shoes white': 0, 'pants white': 1, 'pants blue': 2, 'shoes red': 3, 'shoes black': 4, 'shorts brown': 5, 'shorts white': 6, 'shoes green': 7, 'pants red': 8, 'dress red': 9, 'shirt green': 10, 'dress blue': 11, 'shirt black': 12, 'pants black': 13, 'pants brown': 14, 'dress white': 15, 'pants green': 16, 'shorts green': 17, 'shoes brown': 18, 'dress black': 19, 'shoes blue': 20, 'shorts black': 21, 'shirt blue': 22, 'shorts blue': 23}
510


In [19]:
BATCH_SIZE = 30

train_size = int(0.7 * len(enumerated_dataset))
val_size = int(0.1 * len(enumerated_dataset))
test_size = len(enumerated_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(enumerated_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [20]:
def dimension(N,P,S,F) :
  return (int((N + (2*P) - F)/ S) + 1)

In [21]:
class CNN(nn.Module):
    def __init__(self, kernel_size, stride, padding, pooling_strategy, dropout_rate):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=32,
                               kernel_size = kernel_size,
                               stride=stride,
                               padding=padding)
        dim1 = dimension(110, padding, stride, kernel_size)
        #print(dim1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size, stride, padding)
        dim2 = dimension(dim1, padding, stride, kernel_size)
        #print(dim2)
        if pooling_strategy == "mean" :
          self.pool2d = nn.AvgPool2d(kernel_size=2, stride=2)
        elif pooling_strategy == "max" :
          self.pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        else :
          raise Exception()
        dim3 = int(((dim2-2)/2 +1) * ((dim2-2)/2 +1) * 64) # 'same' = dim3
        #print(dim3)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(dim3, 128)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 30)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2d(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

kernel_size_list = [3, 4, 5]
stride = 1
padding = 0
pooling_strategy_list = ['mean', 'max']
dropout_rate_list = [0, 0.1, 0.25]

In [22]:
# Computing the accuracy of a test set
def evaluate(model, test_loader, error):
#model = mlpinplace=False
    correct = 0
    cur_loss = 0;

    for test_imgs, test_labels in test_loader:
        test_imgs, test_labels = test_imgs.cuda(), test_labels.cuda()
        output = model(test_imgs)
        loss = error(output, test_labels)
        cur_loss+=loss.item()
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    return cur_loss/len(test_loader.dataset), correct/len(test_loader.dataset)



def fit(model, train_loader, error, test_loader, epochs):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)#,lr=0.001, betas=(0.9,0.999))
    model.train()
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        cur_loss = 0;
        correct = 0
        for X_batch, y_batch in train_loader:
            if dev == "cpu":
              X_batch, y_batch = X_batch.cpu(), y_batch.cpu()
            else:
              X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            optimizer.zero_grad()
            output = model(X_batch)
            loss = error(output, y_batch)
            cur_loss+= loss.item()
            loss.backward()
            optimizer.step()

        train_losses.append(cur_loss/len(train_loader.dataset))

        test_loss, _ = evaluate(model, test_loader, error)
        test_losses.append(test_loss)
        print(f'Epoch : {epoch},  train loss:{train_losses[-1]}, test loss:{test_losses[-1]}')

    return train_losses, test_losses

In [23]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [27]:

error = nn.CrossEntropyLoss()
## We use validation data to select the best hyperparameters
## Fewer epochs to speed-up
epochs = 5
### initializing best config
best_accval = 0
best_ks = -1
best_pool = ""
best_dr = -1
for kernel_size in kernel_size_list:
  for pooling_strategy in pooling_strategy_list:
    for dropout_rate in dropout_rate_list:

      model = CNN(kernel_size, stride, padding, pooling_strategy, dropout_rate)
      print(f"kernel size:{kernel_size}, pooling:{pooling_strategy},\
                  dropout rate:{dropout_rate}")

      it = iter(train_loader)
      X_batch, y_batch = next(it)

      if torch.cuda.is_available():
        dev = "cuda:0"
        model.cuda()
      else:
        dev = "cpu"
        model.cpu()
      device = torch.device(dev)


      train_losses, test_losses = fit(model,train_loader, error, val_loader, epochs)

      loss, acc = evaluate(model, train_loader, error)
      print(f"\tTrain Accuracy:{acc}, Train loss:{loss}")

      loss, acc = evaluate(model, val_loader, error)
      print(f"\tValidation Accuracy:{acc}, Validation loss:{loss}")
      if acc > best_accval:
        best_accval = acc
        best_ks = kernel_size
        best_pool = pooling_strategy
        best_dr = dropout_rate

      print("\n\n")

kernel size:3, pooling:mean,                  dropout rate:0


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
print(f"best configuration: kernel size:{best_ks}, pooling:{best_pool},\
                  dropuout rate:{best_dr}")
# Pick the best model and verify its performance on the test set
model = CNN(best_ks, stride, padding, best_pool, best_dr)

#it = iter(train_dataloader)
#X_batch, y_batch = next(it)

if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)

model.cuda()
## we set again more epochs for the final training.
## Note that here we could integrate the validaiton data in the training set
epochs = 15
train_losses, test_losses = fit(model,train_loader, error, test_loader, epochs)

loss, acc = evaluate(model, test_loader, error)
print(f"Test Accuracy:{acc}, Test loss:{loss}")

plt.plot(train_losses,label="training loss")
plt.plot(test_losses,label="testing loss")
plt.legend(fontsize=15)
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.figure()

In [ ]:
# validate and test

loss, acc = evaluate(model, train_loader, error)
print(f"Train Accuracy:{acc}, Train loss:{loss}")

loss, acc = evaluate(model, test_loader, error)
print(f"Test Accuracy:{acc}, Test loss:{loss}")

loss, acc = evaluate(model, val_loader, error)
print(f"Validation Accuracy:{acc}, Validation loss:{loss}")